# Active Inference Ship navigation

This notebooks covers fundamentals of the Active Inference framework implemented with the Bethe Free Energy optimisation with message passing on factor graphs. We use the mountain car problem as an example.

- We refer reader to the [Thijs van de Laar (2019) "Simulating active inference processes by message passing"](https://doi.org/10.3389/frobt.2019.00020) original paper with more in-depth overview and explanation of the active inference agent implementation by message passing.
- The original environment/task description is from [Ueltzhoeffer (2017) "Deep active inference"](https://arxiv.org/abs/1709.02341).

In [779]:
import Pkg; Pkg.activate("."); Pkg.instantiate();

  Activating project at `~/Documents/GitHub/RxInfer.jl/examples`


In [780]:
using RxInfer, Plots

## The mountain and physics 

For the purpose of this example we create a simple mountain valley with hard-coded physics, such that we do not depend on any external complex library. We have several configurable parameters for the experiment:
- Engine-force limit
- Tires friction coefficient

In [781]:
#We need control rudder and accelerator for turning and accelerate. But at first, we only consider turning rate
function ship_dynamic(engine_force_limit=0.1)
    B =[ 0.0,0.0,1.0,0.0 ]
    turning_rate=(a::Real) -> begin
        B*engine_force_limit*tanh(a)
    end
    return (turning_rate)
end

ship_dynamic (generic function with 2 methods)

In [782]:
turning_rate=ship_dynamic(
)

#1244 (generic function with 1 method)

In [783]:
initial_position = [-0.0,0.0,-0.1,3.0]

4-element Vector{Float64}:
 -0.0
  0.0
 -0.1
  3.0

In [784]:
function getAFromState(x,δt)
    A=zeros(4)
    A[1]=x[4]*cos(x[3])*δt
    A[2]=x[4]*sin(x[3])*δt
    return A
end

getAFromState (generic function with 1 method)

In [785]:
getAFromState(initial_position,1)

4-element Vector{Float64}:
  2.9850124958340776
 -0.29950024994048446
  0.0
  0.0

# World - agent interaction

Because the states of the world are unknown to the agent, we wrap them in a comprehension. The comprehension returns only the functions for interacting with the world and not the hidden states. This way, we introduce a stateful world whose states cannot be directly observed.

In [786]:
function create_world(;turning_rate, initial_position = [0.5,0.5,0.1,3.0])

    y_t_min = initial_position
    # y_dot_t_min = initial_velocity
    
    y_t = y_t_min
    # y_dot_t = y_dot_t_min
    
    execute = (rudder::Float64) -> begin
        # Compute next state
        


        # y_dot_t = y_dot_t_min + Fg(y_t_min) + Ff(y_dot_t_min) + Fa(a_t)
        # y_dot_t = getAFromState() + Fg(y_t_min) + Ff(y_dot_t_min) + Fa(a_t)
        y_t = getAFromState(y_t_min,1) + y_t_min + turning_rate(rudder)
    
        # Reset state for next step
        y_t_min = y_t
        # y_dot_t_min = y_dot_t
    end
    
    observe = () -> begin 
        return y_t
    end
        
    return (execute, observe)
end

create_world (generic function with 1 method)

## Naive approach

In this simulation we are going to perform a naive action policy for tight full-power only. In this case, with limited engine power, the agent should not be able to achieve its goal:

In [787]:
N_naive  = 100 # Total simulation time
pi_naive = 0.1 # Naive policy for right full-power only

# Let there be a world
(execute_naive, observe_naive) = create_world(; 
turning_rate = turning_rate,
    initial_position = initial_position, 
);

y_naive = Vector{Vector{Float64}}(undef, N_naive)
for t = 1:N_naive
    execute_naive(pi_naive) # Execute environmental process
    y_naive[t] = observe_naive() # Observe external states
end

x_target=[100.0,100.0,0.0*π,3.0]

animation_naive = @animate for i in 1:N_naive
    # plot(valley_x, valley_y, title = "Naive policy", label = "Landscape", color = "black", size = (800, 400))
    # scatter([y_naive[1][1]], [y_naive[1][2]])
    # plot(y_naive[1][1],y_naive[1:i][2])
    scatter([50], [50],xlims = (-100,100),ylims = (-100,100), label = "Target")
    scatter!([y_naive[i][1]], [y_naive[i][2]],xlims = (-100,100),ylims = (-100,100),label = "ship")
    plot!([y_naive[i][1],y_naive[i][1]+2*y_naive[i][4]*cos(y_naive[i][3])],[y_naive[i][2],y_naive[i][2]+2*y_naive[i][4]*sin(y_naive[i][3])],arrow=true,color=:black,label="")
    
    # scatter!([x_target[1]], [height(x_target[1])], label="goal")   
end

gif(animation_naive, "./ai-mountain-car-naive3.gif", fps = 24, show_msg = false);

[](ai-mountain-car-naive.gif)

# Active inference approach

In the active inference approach we are going to create an agent that models the environment around itself as well as the best possible actions in a probabilistic manner. That should help agent to understand that the brute-force approach is not the most efficient one and hopefully to realise that a little bit of swing is necessary to achieve its goal.

The code in the next block defines the agent's internal beliefs over the external dynamics and its probabilistic model of the environment, which correspond accurately by directly using the functions defined above. We use the `@model` macro from `RxInfer` to define the probabilistic model and the `meta` block to define approximation methods for the nonlinear state-transition functions.

In the model specification we in addition to the current state of the agent we include the beliefs over its future states (up to `T` steps ahead):

In [788]:
hhh = (u::AbstractVector) -> turning_rate(u[1])

#1251 (generic function with 1 method)

In [789]:
turning_rate

#1244 (generic function with 1 method)

In [790]:
hh = (u::AbstractVector) -> turning_rate(u[1])

#1253 (generic function with 1 method)

In [791]:
@model function mountain_car(; T,turning_rate)
    
    # Transition function modeling transition due to gravity and friction
    g = (s_t_min::AbstractVector) -> begin 
        s_t = similar(s_t_min) # Next state
        s_t=getAFromState(s_t_min,1.0) + s_t_min
        return s_t
    end
    
    # Function for modeling engine control
    h = (u::AbstractVector) -> turning_rate(u[1])
    
    # Inverse engine force, from change in state to corresponding engine force
    # h_inv = (delta_s_dot::AbstractVector) -> [atanh(clamp(delta_s_dot[2], -engine_force_limit+1e-3, engine_force_limit-1e-3)/engine_force_limit)] 
    
    # C = constvar([1. 0. 0. 0.;
    #               0. 1. 0. 0.])

    # Internal model perameters
    Gamma = 1e4*diageye(4) # Transition precision
    Theta = 1e-4*diageye(4) # Observation variance
    
    m_s_t_min = datavar(Vector{Float64})
    V_s_t_min = datavar(Matrix{Float64})

    s_t_min ~ MvNormal(mean = m_s_t_min, cov = V_s_t_min)
    s_k_min = s_t_min
    
    m_u = datavar(Vector{Float64}, T)
    V_u = datavar(Matrix{Float64}, T)
    
    m_x = datavar(Vector{Float64}, T)
    V_x = datavar(Matrix{Float64}, T)
    
    u = randomvar(T)
    s = randomvar(T)
    x = randomvar(T)
    
    u_h_k = randomvar(T)
    s_g_k = randomvar(T)
    u_s_sum = randomvar(T)

    for k in 1:T
        u[k] ~ MvNormal(mean = m_u[k], cov = V_u[k])
        u_h_k[k] ~ h(u[k]) where { meta = DeltaMeta(method = Unscented()) }
        s_g_k[k] ~ g(s_k_min) where { meta = DeltaMeta(method = Unscented()) }
        u_s_sum[k] ~ s_g_k[k] + u_h_k[k]
        s[k] ~ MvNormal(mean = u_s_sum[k], precision = Gamma)
        x[k] ~ MvNormal(mean = s[k], cov = Theta)
        x[k] ~ MvNormal(mean = m_x[k], cov = V_x[k]) # goal
        s_k_min = s[k]
    end
    
    return (s, )
end

Because states of the agent are unknown to the world, we wrap them in a comprehension.
The comprehension only returns functions for interacting with the agent.
Internal beliefs cannot be directly observed, and interaction is only allowed through the Markov blanket

In [792]:
# We are going to use some private functionality from ReactiveMP, 
# in the future we should expose a proper API for this
import RxInfer.ReactiveMP: getrecent, messageout

function create_agent(; T = 20, turning_rate, x_target, initial_position)
    Epsilon = fill(0.1, 1, 1)                # Control prior variance
    m_u = Vector{Float64}[ [ 0.0] for k=1:T ] # Set control priors
    V_u = Matrix{Float64}[ Epsilon for k=1:T ]

    Sigma    = 1e-4*diageye(4) # Goal prior variance
    Sigma[3,3]=1e4
    Sigma[4,4]=1e4
    m_x      = [zeros(4) for k=1:T]
    m_x[end]      = x_target
    V_x      = [huge*diageye(4) for k=1:T]
    V_x[end] = Sigma # Set prior to reach goal at t=T

    # Set initial brain state prior
    m_s_t_min = initial_position
    V_s_t_min = tiny * diageye(4)
    
    # Set current inference results
    result = nothing
    pol = zeros(T)

    # The `infer` function is the heart of the agent
    # It calls the `RxInfer.inference` function to perform Bayesian inference by message passing
    infer = (upsilon_t::Float64, y_hat_t::Vector{Float64}) -> begin
        m_u[1] = [ upsilon_t ] # Register action with the generative model
        V_u[1] = fill(tiny, 1, 1) # Clamp control prior to performed action

        m_x[1] = y_hat_t # Register observation with the generative model
        V_x[1] = tiny*diageye(4) # Clamp goal prior to observation

        data = Dict(:m_u       => m_u, 
                    :V_u       => V_u, 
                    :m_x       => m_x, 
                    :V_x       => V_x,
                    :m_s_t_min => m_s_t_min,
                    :V_s_t_min => V_s_t_min)
        
        model  = mountain_car(; T = T, turning_rate) 
        result = inference(model = model, data = data)
        # pol = [mode(result.posteriors[:u][k])[1] for k=1:T]
    end
    
    # The `act` function returns the inferred best possible action
    # act() = pol[1]
    
    act = () -> begin
        if result !== nothing
            return mode(result.posteriors[:u][3])[1] ##
        else
            return 0.0 # Without inference result we return some 'random' action
        end
    end
    
    # The `future` function returns the inferred future states
    future = () -> begin 
        if result !== nothing 
            return getindex.(mode.(result.posteriors[:s]), 1)
        else
            return zeros(T)
        end
    end

    # The `slide` function modifies the `(m_s_t_min, V_s_t_min)` for the next step
    # and shifts (or slides) the array of future goals `(m_x, V_x)` and inferred actions `(m_u, V_u)`
    slide = () -> 
        (s, ) = result.returnval
        
        slide_msg_idx = 3 # This index is model dependend
        (m_s_t_min, V_s_t_min) = mean_cov(getrecent(messageout(s[2], slide_msg_idx)))

        m_u = circshift(m_u, -1)
        m_u[end] = [0.0]
        # m_u = pol
        V_u = circshift(V_u, -1)
        V_u[end] = Epsilon

        m_x = circshift(m_x, -1)
        m_x[end] = x_target
        V_x = circshift(V_x, -1)
        V_x[end] = Sigma
    end

    return (infer, act, slide, future)    
end

create_agent (generic function with 1 method)

In [793]:
(execute_ai, observe_ai) = create_world(;
    turning_rate, 
    initial_position = initial_position
    # initial_velocity = initial_velocity
) # Let there be a world

T_ai = 50

(infer_ai, act_ai, slide_ai, future_ai) = create_agent(; # Let there be an agent
    T  = T_ai, 
    turning_rate=turning_rate,
    x_target = x_target,
    initial_position = initial_position
    # initial_velocity = initial_velocity
) 

N_ai = 100

# Step through experimental protocol
agent_a = Vector{Float64}(undef, N_ai) # Actions
agent_f = Vector{Vector{Float64}}(undef, N_ai) # Predicted future
agent_x = Vector{Vector{Float64}}(undef, N_ai) # Observations
pp=[]
for t=1:N_ai
    agent_a[t] = act_ai()            # Invoke an action from the agent
    agent_f[t] = future_ai()         # Fetch the predicted future states
    execute_ai(agent_a[t])           # The action influences hidden external states
    agent_x[t] = observe_ai()        # Observe the current environmental outcome (update p)
    infer_ai(agent_a[t], agent_x[t]) # Infer beliefs from current model state (update q)
    slide_ai()                       # Prepare for next iteration
end

animation_ai = @animate for i in 1:N_ai
    scatter([x_target[1]], [x_target[2]],xlims = (-200,200),ylims = (-200,200),label = "Target",markersize=6)
    scatter!([agent_x[i][1]], [agent_x[i][2]],xlims = (-200,200),ylims = (-200,200),label = "Ship",markersize=6)
    plot!([agent_x[i][1],agent_x[i][1]+3*agent_x[i][4]*cos(agent_x[i][3])],[agent_x[i][2],agent_x[i][2]+3*agent_x[i][4]*sin(agent_x[i][3])],arrow=true,color=:black,label="")
    
    # scatter!([x_target[1]], [height(x_target[1])], label="goal")   
end

    
gif(animation_ai, "./ai-mountain-car-ai3.gif", fps = 24, show_msg = false);

In [794]:
x_target

4-element Vector{Float64}:
 100.0
 100.0
   0.0
   3.0

In [795]:
agent_a

100-element Vector{Float64}:
   0.0
   0.40890817645181016
   0.46083486081341196
   0.48050139089017263
   0.4833337388408952
   0.4838688947078241
   0.48382289401810846
   0.4831049645778421
   0.48161162161434223
   0.4792951242604246
   ⋮
  31.479219349463268
 -37.82533807000029
   4.28719178364682
   2.3465020457697703
   0.37586424689976705
  -8.523212622911952
   1.733811179222834
   4.012136740008533
 -39.843998570808175

In [796]:
println(agent_a)

[0.0, 0.40890817645181016, 0.46083486081341196, 0.48050139089017263, 0.4833337388408952, 0.4838688947078241, 0.48382289401810846, 0.4831049645778421, 0.48161162161434223, 0.4792951242604246, 0.4761308079331681, 0.4721035919309567, 0.4672040719733284, 0.461427708202163, 0.4547756316801023, 0.4472542420863543, 0.4388757284903514, 0.4296579214827827, 0.4196240882167957, 0.40880304189427114, 0.39722840504729046, 0.3849387407685578, 0.3719773759421766, 0.3583907474780239, 0.3442296069062918, 0.3295472838937424, 0.3143997245979398, 0.29884441008545376, 0.2829407811909678, 0.26674898239442457, 0.25032970475153415, 0.23374448838523984, 0.21705486156453574, 0.20032249649435724, 0.18360945890389282, 0.1669797376732844, 0.1504976780071969, 0.13423212621148065, 0.11825517268432169, 0.10264697146695562, 0.08749722470826443, 0.07291192881241941, 0.059018864748384275, 0.045978181913036736, 0.03399251754941163, 0.0233200787849954, 0.014238202762994898, 0.00665105892469761, -0.006199092934473974, 0.000

In [797]:
execute_ai(agent_a[1])

4-element Vector{Float64}:
 106.5485780937291
 241.45725928705016
   1.9947725480960878
   3.0

As we can see the agent does indeed swing in order to reach its goal. Its interesting though that in the beginning the agent does not attempt to do that but only after some time has passed. That can be explained by the fact that we set `T_ai = 50`, which means that the agent must reach its goal after 50 time steps. In the beginning of the simulation this time horizon appears to be so far in the future that the agent decides not to do anything (in this way the Active Inference agent proved that procrastinating is smart!). After around `30` time steps the goal target becomes closer in time (agent has less than 20 time steps left to achieve the goal) and agent finally decides to act, predicts its future states and realises that in order to achieve its goal it must swing.